In [3]:
pip show langchain_openai

Name: langchain-openai
Version: 0.1.23
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\miniconda\Gen_AI_Automation_Deep_Learning\LangChain_Mastery_Develop_LLM_Apps_with_LangChain_&_Pinecone\venv\Lib\site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get("OPENAI_API_KEY")

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke("Explain Quantum Mechanics in one sentence", model = "gpt-4o-mini")
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the smallest scales, where particles exhibit both wave-like and particle-like properties, and their states are fundamentally probabilistic rather than deterministic.


In [5]:
from langchain.schema import(
    SystemMessage, 
    AIMessage, 
    HumanMessage
)
messages = [
    SystemMessage(content = "You are a physicist and respond only in German"),
    HumanMessage(content="Explain quantum mechanics in one sentence")
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch probabilistische Wellenfunktionen.


## Caching LLM Responses
- Using Langchain's `InMemoryCache()`
- Using `SQLiteCache()`

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")

In [7]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = "Explain quantum mechanics in 5 words"
output = llm.invoke(prompt)

CPU times: total: 31.2 ms
Wall time: 1.32 s


In [8]:
print(output)



Wave-like behavior of subatomic particles.


In [9]:
%%time
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 0 ns


'\n\nWave-like behavior of subatomic particles.'

In [10]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path = ".langchain.db"))

In [11]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: total: 0 ns
Wall time: 717 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired. "

In [12]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: total: 344 ms
Wall time: 361 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired. "

## LLM Streaming

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = "gpt-4o-mini")
prompt = "Write a rock song about the moon and a raven (2 paras)"
print(llm.invoke(prompt).content)

**Title: "Raven's Moonlight"**

(Verse 1)  
Underneath the silver glow, where shadows start to creep,  
A raven calls from the edge of night, secrets it will keep.  
With wings like midnight feathers, it dances on the breeze,  
Whispering to the moonlit sky, a song that brings me to my knees.  
The stars like diamonds flicker, in a canvas dark and deep,  
As the raven soars through the night, while the world around me sleeps.  

(Chorus)  
Oh, raven of the moonlight, take me where you fly,  
Through the mysteries of the night, beneath the endless sky.  
We'll chase the dreams that haunt us, with every haunting cry,  
In the shadows of the moonlight, together we will rise.  
With the fire of the night, and the howl of the wind,  
We'll carve our names in starlight, let the wild ride begin!  


## The main difference between `ChatOpenAI` and `OpenAI` in the langchain_openai library lies in their ***usage and functionality***. `ChatOpenAI` is specifically *designed for chat models*, where the input and output are ***conversational messages***. On the other hand, `OpenAI` is used for *interfacing with language models* (LMS) and is ***NOT specifically tailored for chat interactions***.

In [15]:
for i in llm.stream(prompt):
    print(i.content, end = "", flush=True)

**Verse 1**  
In the shadow of the night, where the silver light glows,  
A raven takes to flight, with secrets only it knows.  
Wings like whispers, cutting through the dark,  
It dances with the moon, leaving behind its mark.  
Howling winds, they call, while the stars ignite the sky,  
Together they weave tales of love and lullabies.  

**Chorus**  
Oh, moonlight dreams and midnight schemes,  
With a raven soaring high, breaking all the seams.  
In a world of shadows, where the wild hearts roam,  
We’ll chase the night together, never far from home.  

## PromptTemplate

In [18]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}'''

prompt_template = PromptTemplate.from_template(template = template)

prompt = prompt_template.format(virus = "Monkey Pox", language = "English")
prompt

'You are an experienced virologist.\nWrite a few sentences about the following virus "Monkey Pox" in English'

In [19]:
llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0)
output = llm.invoke(prompt).content

for i in llm.stream(prompt):
    print(i.content, end = "", flush=True)

Monkeypox is a viral zoonotic disease caused by the monkeypox virus, which is part of the Orthopoxvirus genus, the same family that includes smallpox. Initially identified in laboratory monkeys in 1958, the virus primarily circulates in certain animal populations in Central and West Africa, but human cases have been reported outside these regions. Transmission to humans can occur through direct contact with infected animals, bodily fluids, or contaminated materials, and it can also spread between humans through respiratory droplets or skin lesions. Symptoms typically include fever, rash, and swollen lymph nodes, and while the disease is generally self-limiting, it can lead to severe complications in some cases. Vaccination against smallpox has been shown to provide some cross-protection against monkeypox.

## ChatPromptTemplates

In [20]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]


In [22]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "rank": 1,
            "country": "China",
            "population": "1,439,323,776"
        },
        {
            "rank": 2,
            "country": "India",
            "population": "1,380,004,385"
        },
        {
            "rank": 3,
            "country": "United States",
            "population": "331,002,651"
        },
        {
            "rank": 4,
            "country": "Indonesia",
            "population": "273,523,615"
        },
        {
            "rank": 5,
            "country": "Pakistan",
            "population": "220,892,340"
        }
    ]
}


## Simple Chains

In [4]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model = "gpt-4o-mini")
template = "You are an experienced Virologist. Write a few Sentences about the following virus '{virus}' in '{language}'."
prompt_template = PromptTemplate.from_template(template = template)

chain = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True
)

ouptut = chain.invoke({"virus": "Covid-19", "language": "english"})
print(ouptut)

C:\Users\Akshat Shethia\AppData\Local\Temp\ipykernel_23320\4130418131.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:
You are an experienced Virologist. Write a few Sentences about the following virus 'Covid-19' in 'english'.

> Finished chain.
{'virus': 'Covid-19', 'language': 'english', 'text': 'COVID-19, caused by the novel coronavirus SARS-CoV-2, emerged in late 2019 and quickly led to a global pandemic. The virus primarily spreads through respiratory droplets when an infected person coughs, sneezes, or talks. Symptoms can range from mild respiratory issues to severe pneumonia, and some individuals may remain asymptomatic. Vaccination efforts have been crucial in controlling the spread of the virus and reducing severe illness. Ongoing research continues to monitor variants and their potential impact on transmissibility and vaccine efficacy.'}


In [6]:
template = "What is the capital of {country}? List the top 3 places to visit in that country. Use Bullet Points"
prompt_template = PromptTemplate.from_template(template = template)
chain = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True
)

country = input("Enter the name of a country: ")
output = chain.invoke(country)
print(output["text"])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of India? List the top 3 places to visit in that country. Use Bullet Points

> Finished chain.
The capital of India is New Delhi. Here are the top 3 places to visit in the country:

- **Taj Mahal (Agra)**: A UNESCO World Heritage Site and an iconic symbol of love, this stunning marble mausoleum attracts millions of visitors each year.
  
- **Jaipur (Rajasthan)**: Known as the "Pink City," Jaipur is famous for its majestic forts, palaces, and vibrant markets, including the Hawa Mahal and Amber Fort.
  
- **Varanasi (Uttar Pradesh)**: One of the oldest living cities in the world, Varanasi is a significant cultural and spiritual center, known for its ghats along the Ganges River and its rich traditions.


## Sequential Chains

In [7]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-4o-mini) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4o-mini) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4o-mini', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')



> Entering new SimpleSequentialChain chain...
Certainly! Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. In its simplest form, simple linear regression involves fitting a straight line to a set of data points.

Here's a Python function that implements simple linear regression using the least squares method. This function will calculate the slope and intercept of the regression line and can also make predictions based on the model.

```python
import numpy as np
import matplotlib.pyplot as plt

def linear_regression(x, y):
    """
    Perform simple linear regression on the given data.

    Parameters:
    x (array-like): Independent variable (predictor).
    y (array-like): Dependent variable (response).

    Returns:
    slope (float): Slope of the regression line.
    intercept (float): Intercept of the regression line.
    predictions (array): Predicted values based on the regression line.


In [8]:
print(output["output"])

The provided Python function `linear_regression` implements a simple form of linear regression, which is a core statistical method for modeling the relationship between a dependent variable (often referred to as the response or outcome variable) and one independent variable (the predictor or feature). Let’s dive into the various components of the function, explaining how they operate and their significance.

### Function Breakdown

1. **Imports**:
   ```python
   import numpy as np
   import matplotlib.pyplot as plt
   ```
   - **NumPy**: A fundamental package for scientific computing in Python. It provides support for arrays and mathematical functions.
   - **Matplotlib**: A plotting library in Python for data visualization. It is used here to visualize the data points and the regression line.

2. **Function Definition**:
   ```python
   def linear_regression(x, y):
   ```
   - The function `linear_regression` takes in two arguments: `x` and `y`, which should contain the independent a

##  LangChain Agents

In [10]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip show langchain_experimental

Name: langchain-experimental
Version: 0.0.64
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\miniconda\Gen_AI_Automation_Deep_Learning\LangChain_Mastery_Develop_LLM_Apps_with_LangChain_&_Pinecone\venv\Lib\site-packages
Requires: langchain-community, langchain-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [18]:
prompt = 'Calculate 2 + 2 using Python'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To calculate 2 + 2 using Python, I will execute the addition operation in the Python REPL.  
Action: Python_REPL  
Action Input: print(2 + 2)  
Observation: 4

Thought:I now know the final answer  
Final Answer: 4

> Finished chain.


{'input': 'Calculate 2 + 2 using Python', 'output': '4'}

In [19]:
prompt = 'Calculate the square root of the factorial of 12 using Python'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To calculate the square root of the factorial of 12, I need to first compute the factorial of 12 and then take the square root of that result. I will use the `math` module in Python, which provides a function for calculating factorials and another for calculating square roots.

Action: Python_REPL  
Action Input: `import math; result = math.sqrt(math.factorial(12)); result`  
Observation: 
Thought:I need to print the result to see the output.  
Action: Python_REPL  
Action Input: `import math; result = math.sqrt(math.factorial(12)); print(result)`  
Observation: 21886.105181141756

Thought:I now know the final answer  
Final Answer: 21886.105181141756

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 using Python',
 'output': '21886.105181141756'}

In [16]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True,
    handle_parsing_errors=True  # Add this argument to handle parsing errors
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To calculate the square root of the factorial of 12, I will first compute the factorial of 12, then take the square root of that result. Finally, I will format the output to display it with 4 decimal points. 

Action: Python_REPL  
Action Input: `import math; result = math.sqrt(math.factorial(12)); format_result = format(result, '.4f'); format_result`  
Observation: 
Thought:I need to check the output of the code to see the result of the calculation. 

Action: Python_REPL  
Action Input: `import math; result = math.sqrt(math.factorial(12)); format_result = format(result, '.4f'); format_result`  
Observation: 
Thought:

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I don't know`

## Langchain Tools

In [20]:
pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip show duckduckgo-search

Name: duckduckgo_search
Version: 6.2.11
Summary: Search for words, documents, images, news, maps and text translation using the DuckDuckGo.com search engine.
Home-page: https://github.com/deedy5/duckduckgo_search
Author: deedy5
Author-email: 
License: MIT License
Location: c:\miniconda\Gen_AI_Automation_Deep_Learning\LangChain_Mastery_Develop_LLM_Apps_with_LangChain_&_Pinecone\venv\Lib\site-packages
Requires: click, primp
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
output = search.invoke("Where was Freddie Mercury Born?")
print(output)

Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ... Freddie Mercury was born Farrokh Bulsara in Stone Town in the British protectorate of the Sultanate of Zanzibar, East Africa (now part of Tanzania) on September 5, 1946. When did he get his start in music? Freddie Mercury began his musical career in the late 1960s. He formed his first band, The Hectics, while attending St. Peter's School in ... Freddie Mercury was Britain's first Indian rock star. This fact and the nature of his sexuality were the two areas of his short life about which he would be intentionally obscure. Although he was born on the East African island of Zanzibar on September 5, 1946

In [23]:
search.name, search.description

('duckduckgo_search',
 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

In [24]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.invoke("Freddie Mercury and Queen")
print(output)

[snippet: Freddie Mercury (born September 5, 1946, Stone Town, Zanzibar [now in Tanzania]—died November 24, 1991, Kensington, London, England) was a British rock singer and songwriter whose flamboyant showmanship and powerfully agile vocals, most famously for the band Queen, made him one of rock's most dynamic front men.. Bulsara was born to Parsi parents who had emigrated from India to Zanzibar ..., title: Freddie Mercury | Biography, Parents, Songs, & Facts, link: https://www.britannica.com/biography/Freddie-Mercury], [snippet: Freddie Mercury was a larger than life figure, ... Freddie Mercury Queen. 3 of the Most Historic Moments of Freddie Mercury's Iconic Career. By. Peter Burditt. Updated: September 5, 2024 3:24 pm., title: 3 of the Most Historic Moments of Freddie Mercury's Iconic Career, link: https://americansongwriter.com/3-of-the-most-historic-moments-of-freddie-mercurys-iconic-career/], [snippet: Queen are a British rock band formed in London in 1970 by Freddie Mercury (lea

In [25]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Set region to India
wrapper = DuckDuckGoSearchAPIWrapper(region='in', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Delhi')
print(output)

[snippet: The coldest month is January, with high temperatures in the low 70s F (about 21 °C) and low temperatures in the mid-40s F (about 7 °C). Delhi, city and national capital territory in north-central India. The city of Delhi actually consists of two components: Old Delhi, in the north, the historic city; and New Delhi, in the south, since 1947 ..., title: Delhi | History, Population, Map, & Facts | Britannica, link: https://www.britannica.com/place/Delhi], [snippet: New Delhi, national capital of India. It is situated in the north-central part of the country on the west bank of the Yamuna River, adjacent to and just south of Delhi city (Old Delhi) and within the Delhi national capital territory. In December 1911 King George V of Britain decreed that the capital of British India would be moved from Calcutta ..., title: New Delhi | History, Population, Map, & Facts | Britannica, link: https://www.britannica.com/place/New-Delhi], [snippet: Delhi is both a union territory within Indi

In [26]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: The coldest month is January, with high temperatures in the low 70s F (about 21 °C) and low temperatures in the mid-40s F (about 7 °C). Delhi, city and national capital territory in north-central India. The city of Delhi actually consists of two components: Old Delhi, in the north, the historic city; and New Delhi, in the south, since 1947 ...
Title: Delhi | History, Population, Map, & Facts | Britannica
Link: https://www.britannica.com/place/Delhi

--------------------------------------------------
Snippet: New Delhi, national capital of India. It is situated in the north-central part of the country on the west bank of the Yamuna River, adjacent to and just south of Delhi city (Old Delhi) and within the Delhi national capital territory. In December 1911 King George V of Britain decreed that the capital of British India would be moved from Calcutta ...
Title: New Delhi | History, Population, Map, & Facts | Britannica
Link: https://www.britannica.com/place/New-Delhi

----------

## Creating a ReAct Agent

In [28]:
pip install -q wikipedia

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model (gpt-4o-mini) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

# Define a template for answering questions
template = '''
Answer the following questions in ENGLISH as best you can.
Questions: {q}
'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react')

# displaying information about the react prompt
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# Create tools for the agent

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

c:\miniconda\Gen_AI_Automation_Deep_Learning\LangChain_Mastery_Develop_LLM_Apps_with_LangChain_&_Pinecone\venv\Lib\site-packages\langsmith\client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current prime minister of the UK?'
question = 'Generate the first 20 numbers in the Fibonacci series.'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To generate the first 20 numbers in the Fibonacci series, I can use a simple algorithm that starts with the first two numbers (0 and 1) and then continues to add the last two numbers to get the next one in the series. 

Action: Python REPL  
Action Input: 
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        next_fib = fib_series[-1] + fib_series[-2]
        fib_series.append(next_fib)
    return fib_series

fibonacci(20)
```  I have generated the first 20 numbers in the Fibonacci series. 

Final Answer: The first 20 numbers in the Fibonacci series are: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181].

> Finished chain.


In [8]:
print(output["output"])

The first 20 numbers in the Fibonacci series are: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181].


In [9]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current prime minister of the UK?'
question = 'Tell me about Napoleon Bonaparte early life'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To answer the question about Napoleon Bonaparte's early life, I will look up relevant information on Wikipedia, as it is a reliable source for historical figures.

Action: Wikipedia  
Action Input: "Napoleon Bonaparte early life"  
Page: Napoleon
Summary: Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French military officer and statesman who rose to prominence during the French Revolution and led a series of successful campaigns across Europe during the French Revolutionary and Napoleonic Wars from 1796 to 1815. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 to 1814, and briefly again in 1815.
Born on the island of Corsica to a family of Italian origin, Napoleon moved to mainland France in 1779 and was commissioned as an officer in the French Royal Army in 1785. He supported t

In [10]:
print(output["output"])

Napoleon Bonaparte was born on August 15, 1769, on the island of Corsica to a family of Italian origin. He moved to mainland France in 1779 and was commissioned as an officer in the French Royal Army in 1785. He supported the French Revolution in 1789 and quickly rose through the military ranks, gaining recognition for his victories, including the siege of Toulon in 1793. By 1796, he commanded a military campaign against the Austrians and their Italian allies, becoming a national hero. His early military successes laid the groundwork for his later political power, culminating in his rise to First Consul of the Republic in 1799.
